In [ ]:
import os
from ciao_contrib.runtool import srcflux

# --- User-defined parameters ---

# Base directory where all the OBSID folders are located
base_dir = "/Users/leodrake/Documents/ss433/HRC_2024"

# List of Observation IDs to process
obsids = range(26568, 26580) 

# Updated source position in sexagesimal format
source_pos = "19:11:49.5740,+4:58:57.402"

# --- Paths to Region Files ---
region_dir = os.path.join(base_dir, "reg")
src_region_file = os.path.join(region_dir, "src.reg")
bkg_region_file = os.path.join(region_dir, "bkg.reg")

# Verify that the region files exist before starting
if not os.path.exists(src_region_file) or not os.path.exists(bkg_region_file):
    print("ERROR: Source or background region file not found!")
    print(f"Checked for src: {src_region_file}")
    print(f"Checked for bkg: {bkg_region_file}")
else:
    print("Successfully located source and background region files.")
    print(f"Using source position: {source_pos}")

In [ ]:
print("\n--- Starting batch srcflux processing ---")

for obsid in obsids:
    print(f"\nProcessing OBSID: {obsid}")

    # 1. Construct the full path to the input event file
    evt_file = os.path.join(base_dir, str(obsid), 'repro', f"hrcf{obsid}_repro_evt2_splinecorr.fits")

    # 2. Define the output directory to be 'srcflux' inside the 'repro' folder
    outdir_obs = os.path.join(base_dir, str(obsid), "srcflux", "srcflux")

    # 3. Check if the input file exists to prevent errors
    if not os.path.exists(evt_file):
        print(f"  -> ERROR: Input file not found, skipping.")
        print(f"     (Checked for: {evt_file})")
        continue

    # 4. Run srcflux using the Python interface
    try:
        print(f"  -> Input file: {evt_file}")
        print(f"  -> Source region: {src_region_file}")
        print(f"  -> Bkg region: {bkg_region_file}")

        # Reset srcflux parameters to default before running
        srcflux.punlearn()
        
        # Call the tool with parameters
        srcflux(
            infile=evt_file,
            pos=source_pos,      # Still required, but srcreg takes precedence
            outroot=outdir_obs,   # The 'outroot' where results are saved
            region='optimized',
            #srcreg=src_region_file,
            #bkgreg=bkg_region_file,
            clobber=True,        # Overwrite previous results if they exist
        )
        print(f"  -> SUCCESS: srcflux completed for OBSID {obsid}.")
        print(f"  -> Results are in: {outdir_obs}")

    except Exception as e:
        print(f"  -> ERROR: srcflux failed for OBSID {obsid}.")
        print(f"     CIAO tool returned error: {e}")

print("\n--- All processing complete. ---")